# Grammars and Sentence Parsing, and a Tagging Problem
## Noah Johnson

**Verify that the sentence "I shot an elephant in my pajamas." can be parsed two ways.**

In [1]:
import nltk

groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


We can see that two trees are made out of this sentence. The sentence is structurally ambiguous. The prepositional phrase "in my pajamas" can be included in the noun phrase or the verb phrase. This changes the meaning of the sentence. Either I shot an elephant which was residing inside my pajamas, or I shot an elephant while wearing my pajamas.

## Problem 1

**Confirm that the Groucho grammar thinks the following sentence is illegal: "I shot a lion in my pajamas."**

In [2]:
sent = ['I', 'shot', 'a', 'lion', 'in', 'my', 'pajamas']
try:
    for tree in parser.parse(sent):
        print(tree)
except Exception as e:
    print("ERROR: ", e)

ERROR:  Grammar does not cover some of the input words: "'a', 'lion'".


The sentence is illegal because the words 'a' and 'lion' are not in the vocabulary learned by the grammar, so the parser can't tag the words.

**Alter the grammar so that it allows this sentence.**

In [3]:
new_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my' | 'a'
N -> 'elephant' | 'pajamas' | 'lion'
V -> 'shot'
P -> 'in'
""")

sent = ['I', 'shot', 'a', 'lion', 'in', 'my', 'pajamas']
new_parser = nltk.ChartParser(new_grammar)
for tree in new_parser.parse(sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det a) (N lion)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det a) (N lion) (PP (P in) (NP (Det my) (N pajamas))))))


I added the words 'lion' and 'a' to the new grammar, and now the sentence can be parsed.

## Problem 2

**Alter the grammar so that it allows this sentence: "I aimed at an elephant." This requires new vocabulary words, but it also requires a new kind of verb phrase.**

In [4]:
new_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP | V P NP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot' | 'aimed'
P -> 'in' | 'at'
""")

sent = ['I', 'aimed', 'at', 'an', 'elephant']
new_parser = nltk.ChartParser(new_grammar)
for tree in new_parser.parse(sent):
    print(tree)

(S (NP I) (VP (V aimed) (P at) (NP (Det an) (N elephant))))


The new verb phrase is of the form (Verb Preposition Noun-Phrase).

**Generate 10 possible sentences from this new grammar.**

In [5]:
from nltk.parse.generate import generate

for sentence in generate(new_grammar, n=10):
        print(' '.join(sentence))

an elephant shot an elephant
an elephant shot an pajamas
an elephant shot my elephant
an elephant shot my pajamas
an elephant shot an elephant in an elephant
an elephant shot an elephant in an pajamas
an elephant shot an elephant in my elephant
an elephant shot an elephant in my pajamas
an elephant shot an elephant in an elephant in an elephant
an elephant shot an elephant in an elephant in an pajamas


## Problem 3

**Make up a very different sentence that the grammar thinks is illegal; e.g., "I came, I saw, I conquered."**

After spending quite a while with a long sentence and failing, let's just pick a very simple sentence.

"David please end this eternal nightmare."

In [6]:
# Showing illegality
twain_quote = """
David please end this eternal nightmare
""".split()

try:
    for tree in parser.parse(twain_quote):
        print(tree)
except Exception as e:
    print("ERROR: ", e)

ERROR:  Grammar does not cover some of the input words: "'David', 'please', 'end', 'this', 'eternal', 'nightmare'".


**Use the method of section 3.2 to write a grammar that will accept your sentence, and test it.**

In [7]:
twain_grammar_string = """
S -> NP VP
NP -> Det N | 'David'
VP -> Adv V | VP NP
Det -> 'this'
N -> 'nightmare' | Adj N
Adj -> 'eternal'
V -> 'end'
Adv -> 'please'
"""

with open('twain_grammar.cfg', 'w') as f:
    f.write(twain_grammar_string)

twain_grammar = nltk.data.load('file:twain_grammar.cfg')

twain_parser = nltk.ChartParser(twain_grammar)
for tree in twain_parser.parse(twain_quote):
    print(tree)

(S
  (NP David)
  (VP
    (VP (Adv please) (V end))
    (NP (Det this) (N (Adj eternal) (N nightmare)))))


**Generate 10 possible sentences from your new grammar.**

In [8]:
for sentence in generate(twain_grammar, n=10):
    print(' '.join(sentence))

this nightmare please end
this nightmare please end this nightmare
this nightmare please end this eternal nightmare
this nightmare please end this eternal eternal nightmare
this nightmare please end this eternal eternal eternal nightmare
this nightmare please end this eternal eternal eternal eternal nightmare
this nightmare please end this eternal eternal eternal eternal eternal nightmare
this nightmare please end this eternal eternal eternal eternal eternal eternal nightmare
this nightmare please end this eternal eternal eternal eternal eternal eternal eternal nightmare
this nightmare please end this eternal eternal eternal eternal eternal eternal eternal eternal nightmare


## Problem 4 (open-ended)

**Follow the example in section 6.1 to answer exercise 16.**

### Exercise 16

**Pick some common verbs and complete the following tasks**

In [9]:
my_verbs = ['declined', 'end', 'fell']

**a. Write a program to find those verbs in the Prepositional Phrase Attachment Corpus nltk.corpus.ppattach. Find any cases where the same verb exhibits two different attachments, but where the first noun, or second noun, or preposition, stay unchanged (as we saw in our discussion of syntactic ambiguity in 2).**

In [10]:
# What does the training set look like?
entries = nltk.corpus.ppattach.attachments('training')
for entry in entries:
    if entry.verb in my_verbs:
        print(entry)

PPAttachment(sent='389', verb='fell', noun1='%', prep='to', noun2='billion', attachment='V')
PPAttachment(sent='1302', verb='end', noun1='December', prep='with', noun2='balance', attachment='V')
PPAttachment(sent='1541', verb='fell', noun1='total', prep='of', noun2='points', attachment='N')
PPAttachment(sent='1641', verb='fell', noun1='%', prep='in', noun2='1977', attachment='V')
PPAttachment(sent='2808', verb='fell', noun1='%', prep='to', noun2='million', attachment='V')
PPAttachment(sent='2811', verb='fell', noun1='%', prep='to', noun2='million', attachment='V')
PPAttachment(sent='2813', verb='fell', noun1='%', prep='to', noun2='million', attachment='V')
PPAttachment(sent='2924', verb='fell', noun1='1.125', prep='to', noun2='13.625', attachment='V')
PPAttachment(sent='2925', verb='fell', noun1='%', prep='to', noun2='%', attachment='V')
PPAttachment(sent='2934', verb='fell', noun1='points', prep='to', noun2='35500.64', attachment='V')
PPAttachment(sent='2952', verb='fell', noun1='yen'

PPAttachment(sent='24839', verb='fell', noun1='7', prep='to', noun2='44', attachment='V')
PPAttachment(sent='24851', verb='fell', noun1='3.10', prep='to', noun2='376.36', attachment='V')
PPAttachment(sent='24864', verb='fell', noun1='%', prep='to', noun2='million', attachment='V')
PPAttachment(sent='24870', verb='fell', noun1='%', prep='to', noun2='billion', attachment='V')
PPAttachment(sent='24889', verb='fell', noun1='points', prep='on', noun2='news', attachment='V')
PPAttachment(sent='24898', verb='fell', noun1='1', prep='to', noun2='point', attachment='V')
PPAttachment(sent='25206', verb='declined', noun1='%', prep='to', noun2='billion', attachment='V')
PPAttachment(sent='25222', verb='declined', noun1='million', prep='to', noun2='billion', attachment='V')
PPAttachment(sent='25228', verb='fell', noun1='%', prep='to', noun2='million', attachment='V')
PPAttachment(sent='25245', verb='end', noun1='involvement', prep='in', noun2='mining', attachment='N')
PPAttachment(sent='25299', verb

Ok, so the entries is a list of PPAttachment objects, which have sentence_id, verb, noun1, preposition, noun2, and attachment member variables. This really looks like it should be a data frame. Let's convert it. If you don't have pandas installed on your user, you should go install it (perhaps for everyone with sudo).

In [11]:
import pandas as pd

sent_ids = []
verbs = []
noun1s = []
preps = []
noun2s = []
attachments = []
for entry in entries:
    if entry.verb in my_verbs:
        sent_ids.append(entry.sent)
        verbs.append(entry.verb)
        noun1s.append(entry.noun1)
        preps.append(entry.prep)
        noun2s.append(entry.noun2)
        attachments.append(entry.attachment)
        
df = pd.DataFrame(dict(sent_id=sent_ids, verb=verbs, noun1=noun1s,
                       prep=preps, noun2=noun2s, attachment=attachments))
print(df.head())

  attachment     noun1    noun2  prep sent_id  verb
0          V         %  billion    to     389  fell
1          V  December  balance  with    1302   end
2          N     total   points    of    1541  fell
3          V         %     1977    in    1641  fell
4          V         %  million    to    2808  fell


In [12]:
for label in ['noun1', 'prep', 'noun2']:
    print("Example entries with same verb and %s but different attachments:" % label)
    ambiguous_entries = df[df.groupby(['verb', label])['attachment'].transform('nunique') > 1].sort_values(['verb', label])
    print(ambiguous_entries[['verb', 'noun1', 'prep', 'noun2', 'attachment']])
    print('')

Example entries with same verb and noun1 but different attachments:
         verb       noun1     prep      noun2 attachment
14   declined           %       to    million          V
35   declined           %       to      units          V
59   declined           %       in     period          V
121  declined           %       to    billion          V
153  declined           %     from       year          V
160  declined           %       in       week          N
176  declined           %       in  September          V
186  declined           %       to    billion          V
196  declined           %  because       sale          V
61        end  cease-fire     with     rebels          N
62        end  cease-fire       in   response          V
31       fell           3       to         99          V
63       fell           3       to      point          N
66       fell           3       to         49          V
98       fell           3       to         12          V
103      fell       

**b. Devise CFG grammar productions to cover some of these cases.**

???

## Problem 5

**Use the method of Church to find the most likely parsing of the sentence, "I tried to please you." Show each step, especially the last step ("I") where the use of the trigram model for context probability allows the algorithm to prune the search to run in linear time in the length of the sentence.**

First I'll get the tagged counts for each word that shows up in our sentence, within the Tagged Brown Corpus.

In [13]:
from nltk.corpus import brown

sent = "I tried to please you".split()

# Keys are (word, part-of-speech), values
# are frequency counts of that pair
word_tag_counts = {}
for word, pos in brown.tagged_words():
    
    if word in sent:
        word_tag_counts[(word, pos)] = word_tag_counts.get((word, pos), 0) + 1

print(word_tag_counts)

{('I', 'NP'): 1, ('I', 'PPSS-HL'): 2, ('you', 'PPSS'): 1864, ('I', 'NIL'): 1, ('please', 'VB'): 37, ('tried', 'VBN'): 50, ('you', 'PPSS-HL'): 1, ('to', 'NPS'): 1, ('to', 'TO-HL'): 50, ('to', 'TO-NC'): 13, ('to', 'IN-TL'): 12, ('I', 'NN-TL'): 1, ('you', 'PPO'): 889, ('please', 'UH'): 8, ('to', 'QL'): 1, ('to', 'NIL'): 3, ('to', 'IN-NC'): 8, ('tried', 'VBD'): 120, ('you', 'PPO-HL'): 1, ('I', 'PPSS'): 5129, ('you', 'PPO-NC'): 4, ('to', 'TO'): 14679, ('you', 'PPSS-NC'): 7, ('to', 'IN'): 10903, ('I', 'NN'): 3, ('to', 'IN-HL'): 62, ('I', 'PPSS-NC'): 24}


Ok, so we counted (word, part-of-speech) pairs. Now each of those entries can be cataloged in a data frame.

In [14]:
# Create part-of-speech frequencies data frame in long-form
pos_frequencies = pd.DataFrame(columns=['word', 'pos', 'count'])
for i, key in enumerate(word_tag_counts.keys()):
    word = key[0]
    pos = key[1]
    pos_frequencies.loc[i] = [word, pos, word_tag_counts[(word, pos)]]  # add row to data frame
    
print(pos_frequencies.head())

     word      pos count
0       I       NP     1
1       I  PPSS-HL     2
2     you     PPSS  1864
3       I      NIL     1
4  please       VB    37


We're going to need lexical probabilities, so let's compute those now. For each (word, part-of-speech) pair, we'll divide the count by the count of that word (which is the sum of counts over all parts of speech for that word). Also, since these probabilities can get small, we'll take the log of the probabilities.

In [15]:
import numpy as np

# Calculate lexical probabilities by taking count(word,pos) / count(word)
pos_frequencies['lexical_probs'] = pos_frequencies.groupby('word')['count'].transform(lambda x: x / x.sum())

# Get log probs
pos_frequencies['lexical_log_probs'] = np.log(pos_frequencies['lexical_probs'].astype(np.float64))

print(pos_frequencies.head())

     word      pos count lexical_probs  lexical_log_probs
0       I       NP     1   0.000193761          -8.548886
1       I  PPSS-HL     2   0.000387522          -7.855738
2     you     PPSS  1864      0.673897          -0.394678
3       I      NIL     1   0.000193761          -8.548886
4  please       VB    37      0.822222          -0.195745


Awesome. We are going to need unigram, bigram and trigram counts for parts of speech, in order to compute the contextual probabilities. I can't easily find a simple nltk way to do this, so we'll have to manually build these. First build the unigram table.

In [16]:
# parts of speech that we're interested in (that could occur in our sentence)
all_pos = set(pos_frequencies['pos'].unique())

# (pos) -> unigram_count
unigram_pos_counts = {pos: 1 for pos in all_pos}  # laplace smoothed

# handle start tag
unigram_pos_counts[''] = 1
    
for word, pos in brown.tagged_words():

    # Only count if this part of speech is one we're interested in
    if pos in all_pos:
        unigram_pos_counts[pos] += 1
        
    # Naive approach - using periods as start tags
    if pos == '.':
        unigram_pos_counts[''] += 1

print(unigram_pos_counts)

{'': 60639, 'PPO': 11182, 'PPSS': 13803, 'NPS': 1276, 'PPO-HL': 6, 'NP': 34477, 'IN': 120558, 'IN-NC': 42, 'IN-TL': 1478, 'TO': 14919, 'NN-TL': 13373, 'VBD': 26168, 'VBN': 29187, 'IN-HL': 509, 'UH': 609, 'NIL': 158, 'PPSS-HL': 26, 'PPSS-NC': 32, 'NN': 152471, 'PPO-NC': 10, 'VB': 33694, 'QL': 8736, 'TO-NC': 14, 'TO-HL': 56}


Then the bigram table.

In [17]:
# (pos1, pos2) -> bigram_count
bigram_pos_counts = {(pos1, pos2): 1 for pos1 in all_pos for pos2 in all_pos}  # laplace smoothed

# handle start tags
start_bigrams = [('', pos) for pos in all_pos]
for start_bigram in start_bigrams:
    bigram_pos_counts[start_bigram] = 1
    
for t1, t2 in zip(brown.tagged_words()[:-1], brown.tagged_words()[1:]):
    pos1 = t1[1]
    pos2 = t2[1]
    
    # Only count bigram if both parts of speech are ones we're interested in
    if pos1 in all_pos and pos2 in all_pos:
        bigram_pos_counts[(pos1, pos2)] += 1
    # Naive approach - using periods as start tags
    elif pos1 == '.' and pos2 in all_pos:
        bigram_pos_counts[('', pos2)] += 1

print(bigram_pos_counts)

{('PPO', 'VB'): 438, ('PPO-HL', 'PPO'): 1, ('IN-HL', 'NP'): 1, ('PPSS-HL', 'VBD'): 1, ('PPO-NC', 'PPO-HL'): 1, ('PPO', 'TO-NC'): 1, ('NPS', 'VBN'): 7, ('TO', 'NN-TL'): 1, ('IN-TL', 'PPSS-NC'): 1, ('NIL', 'IN-NC'): 1, ('UH', 'NIL'): 1, ('PPSS-HL', 'NN'): 1, ('IN-NC', 'IN-TL'): 1, ('NP', 'TO-NC'): 1, ('NN-TL', 'NN'): 522, ('', 'NN'): 1246, ('NN-TL', 'IN-HL'): 3, ('NN-TL', 'IN'): 852, ('NN-TL', 'PPSS'): 17, ('PPSS-HL', 'UH'): 1, ('TO', 'NIL'): 1, ('TO-HL', 'NIL'): 1, ('IN-NC', 'PPSS-HL'): 1, ('IN', 'VBN'): 616, ('NIL', 'NPS'): 1, ('VBD', 'TO'): 1220, ('IN-NC', 'VBD'): 1, ('PPO-NC', 'PPSS'): 1, ('PPSS', 'PPSS-NC'): 1, ('TO-HL', 'NN-TL'): 1, ('IN-HL', 'PPO-NC'): 1, ('TO-NC', 'TO'): 1, ('PPSS-NC', 'NN-TL'): 1, ('IN-HL', 'IN'): 1, ('IN-NC', 'TO'): 1, ('VBN', 'VBD'): 15, ('TO-HL', 'PPO'): 1, ('NPS', 'PPO'): 1, ('TO-NC', 'VBN'): 1, ('UH', 'NN-TL'): 2, ('VB', 'PPSS-HL'): 1, ('PPSS-HL', 'PPSS-HL'): 1, ('PPO', 'PPSS-HL'): 1, ('TO-NC', 'PPO-NC'): 1, ('NP', 'VBD'): 3828, ('NN', 'NPS'): 15, ('PPSS-HL

And now the trigram table.

In [18]:
# (pos1, pos2, pos3) -> trigram_count
trigram_pos_counts = {(pos1, pos2, pos3): 1 for pos1 in all_pos for pos2 in all_pos for pos3 in all_pos}  # laplace smoothed

# handle start tags
single_start_trigrams = [('', pos1, pos2) for pos1 in all_pos for pos2 in all_pos]
for start_trigram in single_start_trigrams:
    trigram_pos_counts[start_trigram] = 1
    
double_start_trigrams = [('', '', pos) for pos in all_pos]
for start_trigram in double_start_trigrams:
    trigram_pos_counts[start_trigram] = 1
    
for t1, t2, t3 in zip(brown.tagged_words()[:-2], brown.tagged_words()[1:-1], brown.tagged_words()[2:]):
    pos1 = t1[1]
    pos2 = t2[1]
    pos3 = t3[1]
    
    # Only count trigram if all three parts of speech are ones we're interested in
    if pos1 in all_pos and pos2 in all_pos and pos3 in all_pos:
        trigram_pos_counts[(pos1, pos2, pos3)] += 1  
    # Naive approach - using periods as start tags
    elif pos1 == '.' and pos2 in all_pos and pos3 in all_pos:
        trigram_pos_counts[('', pos2, pos3)] += 1
    elif pos2 == '.' and pos3 in all_pos:
        trigram_pos_counts[('', '', pos3)] += 1

print(trigram_pos_counts)

{('NN-TL', 'PPO-HL', 'IN-NC'): 1, ('IN', 'PPSS-HL', 'IN-TL'): 1, ('PPO-HL', 'PPSS', 'PPSS'): 1, ('IN-TL', 'VBN', 'PPO-HL'): 1, ('VB', 'IN', 'IN-NC'): 1, ('PPSS-HL', 'NP', 'IN'): 1, ('PPSS', 'VBD', 'QL'): 28, ('NP', 'IN', 'TO'): 1, ('PPO-NC', 'IN-HL', 'NN-TL'): 1, ('PPSS-NC', 'PPSS', 'IN-TL'): 1, ('VBD', 'PPO', 'PPSS'): 13, ('QL', 'TO-NC', 'PPO-HL'): 1, ('NP', 'PPSS-NC', 'PPO'): 1, ('VBN', 'QL', 'TO-NC'): 1, ('NN-TL', 'PPSS-NC', 'VBN'): 1, ('UH', 'IN', 'PPSS'): 1, ('NP', 'PPSS', 'IN'): 1, ('VB', 'TO-HL', 'IN-TL'): 1, ('PPO-NC', 'VBD', 'VBD'): 1, ('NPS', 'NIL', 'VB'): 1, ('QL', 'TO', 'NPS'): 1, ('TO-HL', 'QL', 'IN-NC'): 1, ('VBD', 'VBD', 'PPO'): 1, ('PPSS-NC', 'PPSS', 'UH'): 1, ('VB', 'VBD', 'IN'): 2, ('TO-NC', 'TO-NC', 'NN-TL'): 1, ('PPO-NC', 'TO', 'NN-TL'): 1, ('NN', 'TO-NC', 'PPO-NC'): 1, ('NP', 'NPS', 'PPO-NC'): 1, ('TO-HL', 'NPS', 'PPO'): 1, ('VBD', 'TO', 'NIL'): 1, ('IN-HL', 'PPO-HL', 'UH'): 1, ('IN', 'NPS', 'VB'): 2, ('VB', 'PPSS-NC', 'PPSS-NC'): 1, ('IN-HL', 'PPO-HL', 'PPO-NC'): 

Ok. Now we will be able to build the contextual probabilities when we need them. The contextual probability is the probability of observing part of speech X given the following two parts of speech Y and Z. We will estimate this by dividing the count of trigram XYZ by the count of bigram YZ. Similarly for the case when X is followed only by Y, we will estimate the contextual probability by dividing the count of bigram XY by the count of unigram Y.

Now we have all the pieces necessary for generating scores for partial tagged sequences.

In [19]:
def compute_contextual_log_probability(pos_sequence):
    """ Take in an iterable listing a sequence of parts of speech. """
    
    pos_window = pos_sequence[:3]
    
    num_pos_in_window = len(pos_window)
    
    if num_pos_in_window == 1:  # contextual prob of a single pos gives no info, so prob = 1, logprob = 0
        return 0
    elif num_pos_in_window == 2:
        X = pos_sequence[0]
        Y = pos_sequence[1]
        
        bigram_count = bigram_pos_counts[(X, Y)]
        unigram_count = unigram_pos_counts[Y]
        
        return np.log(bigram_count / unigram_count)
    else:  # trigram
        X = pos_sequence[0]
        Y = pos_sequence[1]
        Z = pos_sequence[2]
        
        trigram_count = trigram_pos_counts[(X, Y, Z)]
        bigram_count = bigram_pos_counts[(Y, Z)]
        
        return np.log(trigram_count / bigram_count)

test_sequences = [['NP'],
                  ['NP', 'VB'],
                  ['NP', 'VB', 'PPO']]

for test_sequence in test_sequences:
    print(test_sequence, ": ", compute_contextual_log_probability(test_sequence))

['NP'] :  0
['NP', 'VB'] :  -5.221068371869354
['NP', 'VB', 'PPO'] :  -4.855928904335275


We made this convenient function that we can vectorize.

Now we can start tagging our sentence. Starting with the last word, 'you', let's get our partial sequence scores.

In [20]:
# Get all assignments of parts of speech to 'you', and their lexical log probabilities
you_assignments = pos_frequencies[pos_frequencies.word == 'you'][['lexical_log_probs', 'pos']]

# This data frame will hold all the current candidate best assignments for partial sequences
partial_sequences = pd.DataFrame(you_assignments['pos'])

# Glue code
partial_sequences['pos_list'] = partial_sequences['pos'].apply(lambda s: (s,))  # make tuple of length 1

# Multiply together lexical probability with contextual probability to get score of sequence
# Adding in log space is multiplying in probability space
partial_sequences['score'] = you_assignments['lexical_log_probs'] + \
                             partial_sequences['pos_list'].apply(compute_contextual_log_probability)

partial_sequences = partial_sequences[['score', 'pos_list']].reset_index(drop=True)
print(partial_sequences)

      score    pos_list
0 -0.394678     (PPSS,)
1 -7.925158  (PPSS-HL,)
2 -1.135060      (PPO,)
3 -7.925158   (PPO-HL,)
4 -6.538863   (PPO-NC,)
5 -5.979247  (PPSS-NC,)


The next word of our sentence is 'please'.

In [21]:
# Get all assignments of parts of speech to 'please', and their lexical log probabilities
please_assignments = pos_frequencies[pos_frequencies.word == 'please'][['lexical_log_probs', 'pos']]

# Compute the cross product (all combinations) of the two series of parts of speech,
# to generate all part of speech assignments for the two-word sequence 'please you'
cross_product = partial_sequences.assign(tmp=1).merge(please_assignments.assign(tmp=1),
                                                      how='outer', on='tmp').drop('tmp', axis=1)

# Combine the parts of speech with the splat operator
cross_product['pos_list'] = cross_product.apply(lambda row: (row['pos'], *row['pos_list']), axis=1)

# Add new probability to old
cross_product['score'] += cross_product['lexical_log_probs'] + \
                          cross_product['pos_list'].apply(compute_contextual_log_probability)

# Throw away the log prob columns we no longer need
partial_sequences = cross_product[['score', 'pos_list']].reset_index(drop=True)
print(partial_sequences)

        score       pos_list
0   -4.663478     (VB, PPSS)
1   -9.708630     (UH, PPSS)
2  -11.378999  (VB, PPSS-HL)
3  -12.910475  (UH, PPSS-HL)
4   -2.618883      (VB, PPO)
5  -12.184342      (UH, PPO)
6   -9.912662   (VB, PPO-HL)
7  -11.444138   (UH, PPO-HL)
8   -9.037193   (VB, PPO-NC)
9  -10.568669   (UH, PPO-NC)
10  -8.947581  (VB, PPSS-NC)
11 -11.172204  (UH, PPSS-NC)


The next word of our sentence is 'to'.

In [22]:
# Get all assignments of parts of speech to 'to', and their lexical log probabilities
to_assignments = pos_frequencies[pos_frequencies.word == 'to'][['lexical_log_probs', 'pos']]

# Compute the cross product (all combinations) of the two series of parts of speech,
# to generate all part of speech assignments for the three-word sequence 'to please you'
cross_product = partial_sequences.assign(tmp=1).merge(to_assignments.assign(tmp=1),
                                                      how='outer', on='tmp').drop('tmp', axis=1)

# Combine the parts of speech
cross_product['pos_list'] = cross_product.apply(lambda row: (row['pos'], *row['pos_list']), axis=1)

# Add new probability to old
cross_product['score'] += cross_product['lexical_log_probs'] + \
                          cross_product['pos_list'].apply(compute_contextual_log_probability)

# Throw away the log prob columns we no longer need
partial_sequences = cross_product[['score', 'pos_list']].reset_index(drop=True)
print(partial_sequences)

         score              pos_list
0   -20.278554       (NPS, VB, PPSS)
1   -16.366531     (TO-HL, VB, PPSS)
2   -17.713605     (TO-NC, VB, PPSS)
3   -17.793647     (IN-TL, VB, PPSS)
4   -20.278554        (QL, VB, PPSS)
5   -19.179942       (NIL, VB, PPSS)
6   -18.199112     (IN-NC, VB, PPSS)
7    -7.739942        (TO, VB, PPSS)
8   -10.981761        (IN, VB, PPSS)
9   -16.151420     (IN-HL, VB, PPSS)
10  -21.810030       (NPS, UH, PPSS)
11  -17.898007     (TO-HL, UH, PPSS)
12  -19.245081     (TO-NC, UH, PPSS)
13  -19.325124     (IN-TL, UH, PPSS)
14  -21.810030        (QL, UH, PPSS)
15  -20.711418       (NIL, UH, PPSS)
16  -19.730589     (IN-NC, UH, PPSS)
17  -12.215857        (TO, UH, PPSS)
18  -12.513237        (IN, UH, PPSS)
19  -17.682896     (IN-HL, UH, PPSS)
20  -21.534489    (NPS, VB, PPSS-HL)
21  -17.622466  (TO-HL, VB, PPSS-HL)
22  -18.969540  (TO-NC, VB, PPSS-HL)
23  -19.049583  (IN-TL, VB, PPSS-HL)
24  -21.534489     (QL, VB, PPSS-HL)
25  -20.435877    (NIL, VB, PPSS-HL)
2

The next word of our sentence is 'tried'.

In [23]:
# Get all assignments of parts of speech to 'tried', and their lexical log probabilities
tried_assignments = pos_frequencies[pos_frequencies.word == 'tried'][['lexical_log_probs', 'pos']]

# Compute the cross product (all combinations) of the two series of parts of speech,
# to generate all part of speech assignments for the four-word sequence 'tried to please you'
cross_product = partial_sequences.assign(tmp=1).merge(tried_assignments.assign(tmp=1),
                                                      how='outer', on='tmp').drop('tmp', axis=1)

# Combine the parts of speech
cross_product['pos_list'] = cross_product.apply(lambda row: (row['pos'], *row['pos_list']), axis=1)

# Add new probability to old
cross_product['score'] += cross_product['lexical_log_probs'] + \
                          cross_product['pos_list'].apply(compute_contextual_log_probability)

# Throw away the log prob columns we no longer need
partial_sequences = cross_product[['score', 'pos_list']].reset_index(drop=True)
print(partial_sequences)

         score                   pos_list
0   -24.983569       (VBN, NPS, VB, PPSS)
1   -24.801248       (VBD, NPS, VB, PPSS)
2   -17.590306     (VBN, TO-HL, VB, PPSS)
3   -16.714838     (VBD, TO-HL, VB, PPSS)
4   -18.937380     (VBN, TO-NC, VB, PPSS)
5   -18.061911     (VBD, TO-NC, VB, PPSS)
6   -19.017423     (VBN, IN-TL, VB, PPSS)
7   -18.141954     (VBD, IN-TL, VB, PPSS)
8   -23.294089        (VBN, QL, VB, PPSS)
9   -22.418620        (VBD, QL, VB, PPSS)
10  -20.403717       (VBN, NIL, VB, PPSS)
11  -19.528248       (VBD, NIL, VB, PPSS)
12  -19.422888     (VBN, IN-NC, VB, PPSS)
13  -18.547419     (VBD, IN-NC, VB, PPSS)
14  -11.236804        (VBN, TO, VB, PPSS)
15  -10.612311        (VBD, TO, VB, PPSS)
16  -14.556911        (VBN, IN, VB, PPSS)
17  -14.374590        (VBD, IN, VB, PPSS)
18  -17.375195     (VBN, IN-HL, VB, PPSS)
19  -16.499726     (VBD, IN-HL, VB, PPSS)
20  -23.033806       (VBN, NPS, UH, PPSS)
21  -22.158337       (VBD, NPS, UH, PPSS)
22  -19.121783     (VBN, TO-HL, UH

The number of partial sequences is blowing up exponentially. Luckily, we have reached the limit of our three-tag window, and can now remove paths.

In [24]:
def filter_out_sequences(partial_sequences):
    
    # Get the two front parts of speech
    partial_sequences['two_most_recent_pos'] = partial_sequences['pos_list'].apply(lambda l: l[:2])
    #print(partial_sequences)

    # Grouping by the two front parts of speech, find the rows with the maximum score for each group
    keep_idx = partial_sequences.groupby('two_most_recent_pos')['score'].idxmax().values
    #print(keep_idx)

    # Filter and keep only those rows
    partial_sequences = partial_sequences.loc[keep_idx]
    #print(partial_sequences)

    partial_sequences = partial_sequences[['score', 'pos_list']].reset_index(drop=True)
    return partial_sequences

partial_sequences = filter_out_sequences(partial_sequences)
print(partial_sequences)

        score                  pos_list
0  -13.288719     (VBD, IN, VB, PPO-NC)
1  -15.413856  (VBD, IN-HL, VB, PPO-NC)
2  -17.461549  (VBD, IN-NC, VB, PPO-NC)
3  -17.056083  (VBD, IN-TL, VB, PPO-NC)
4  -18.442378    (VBD, NIL, VB, PPO-NC)
5  -21.072467    (VBD, NPS, UH, PPO-NC)
6  -21.072467     (VBD, QL, UH, PPO-NC)
7   -7.157035        (VBD, TO, VB, PPO)
8  -15.628967  (VBD, TO-HL, VB, PPO-NC)
9  -16.976041  (VBD, TO-NC, VB, PPO-NC)
10 -13.471041     (VBN, IN, VB, PPO-NC)
11 -16.289325  (VBN, IN-HL, VB, PPO-NC)
12 -18.337017  (VBN, IN-NC, VB, PPO-NC)
13 -17.931552  (VBN, IN-TL, VB, PPO-NC)
14 -19.317847    (VBN, NIL, VB, PPO-NC)
15 -21.947935    (VBN, NPS, UH, PPO-NC)
16 -21.947935     (VBN, QL, UH, PPO-NC)
17  -7.781528        (VBN, TO, VB, PPO)
18 -16.504436  (VBN, TO-HL, VB, PPO-NC)
19 -17.851510  (VBN, TO-NC, VB, PPO-NC)


Ok, now the list is much shorter! Let's keep going. The next and final word of our sentence is 'I'.

In [25]:
# Get all assignments of parts of speech to 'I', and their lexical log probabilities
I_assignments = pos_frequencies[pos_frequencies.word == 'I'][['lexical_log_probs', 'pos']]

# Compute the cross product (all combinations) of the two series of parts of speech,
# to generate all part of speech assignments for the four-word sequence 'I tried to please you'
cross_product = partial_sequences.assign(tmp=1).merge(I_assignments.assign(tmp=1),
                                                      how='outer', on='tmp').drop('tmp', axis=1)

# Combine the parts of speech
cross_product['pos_list'] = cross_product.apply(lambda row: (row['pos'], *row['pos_list']), axis=1)

# Add new probability to old
cross_product['score'] += cross_product['lexical_log_probs'] + \
                          cross_product['pos_list'].apply(compute_contextual_log_probability)

# Throw away the log prob columns we no longer need
partial_sequences = cross_product[['score', 'pos_list']].reset_index(drop=True)
print(partial_sequences)

         score                           pos_list
0   -23.961374          (NP, VBD, IN, VB, PPO-NC)
1   -29.529718     (PPSS-HL, VBD, IN, VB, PPO-NC)
2   -30.222865         (NIL, VBD, IN, VB, PPO-NC)
3   -26.111992       (NN-TL, VBD, IN, VB, PPO-NC)
4   -15.577641        (PPSS, VBD, IN, VB, PPO-NC)
5   -22.653454          (NN, VBD, IN, VB, PPO-NC)
6   -27.044812     (PPSS-NC, VBD, IN, VB, PPO-NC)
7   -23.962741       (NP, VBD, IN-HL, VB, PPO-NC)
8   -23.269594  (PPSS-HL, VBD, IN-HL, VB, PPO-NC)
9   -23.962741      (NIL, VBD, IN-HL, VB, PPO-NC)
10  -23.962741    (NN-TL, VBD, IN-HL, VB, PPO-NC)
11  -15.420075     (PPSS, VBD, IN-HL, VB, PPO-NC)
12  -22.864129       (NN, VBD, IN-HL, VB, PPO-NC)
13  -20.784688  (PPSS-NC, VBD, IN-HL, VB, PPO-NC)
14  -26.010434       (NP, VBD, IN-NC, VB, PPO-NC)
15  -25.317287  (PPSS-HL, VBD, IN-NC, VB, PPO-NC)
16  -26.010434      (NIL, VBD, IN-NC, VB, PPO-NC)
17  -26.010434    (NN-TL, VBD, IN-NC, VB, PPO-NC)
18  -17.467768     (PPSS, VBD, IN-NC, VB, PPO-NC)


Once again, filter out assignments which score less than some other path in their window group.

In [26]:
partial_sequences = filter_out_sequences(partial_sequences)
print(partial_sequences)

        score                     pos_list
0  -22.812527      (NIL, VBD, TO, VB, PPO)
1  -23.669951      (NIL, VBN, TO, VB, PPO)
2  -16.779441       (NN, VBD, TO, VB, PPO)
3  -18.428204       (NN, VBN, TO, VB, PPO)
4  -19.979314    (NN-TL, VBD, TO, VB, PPO)
5  -22.976804    (NN-TL, VBN, TO, VB, PPO)
6  -17.870884       (NP, VBD, TO, VB, PPO)
7  -22.571339       (NP, VBN, TO, VB, PPO)
8   -9.176111     (PPSS, VBD, TO, VB, PPO)
9  -15.127285     (PPSS, VBN, TO, VB, PPO)
10 -22.119380  (PPSS-HL, VBD, TO, VB, PPO)
11 -22.976804  (PPSS-HL, VBN, TO, VB, PPO)
12 -19.634473  (PPSS-NC, VBD, TO, VB, PPO)
13 -20.491897  (PPSS-NC, VBN, TO, VB, PPO)


Now just continue with a couple of blank parts of speech for the words out of range. The lexical probability of a word out of range should be 1, i.e. should not affect the scores. However the look-ahead contextual probability could still change the scores based on which parts of speech are most likely to start a sentence. Earlier when computing the n-gram tables, I made some simple assumptions using a period '.' part-of-speech, which is in the tagged corpus, as a start symbol. This gives us an estimate of the contextual probabilities that we can use now.

In [27]:
# No need to compute a cross-product, since there is only one possible part of speech for a word out of range: ''
# Prepend a blank part-of-speech to all the part of speech sequences
partial_sequences['pos_list'] = partial_sequences.apply(lambda row: ('', *row['pos_list']), axis=1)

# Add contextual log probabilities to old
partial_sequences['score'] += partial_sequences['pos_list'].apply(compute_contextual_log_probability)

#print(partial_sequences)

# Filter out bad paths
partial_sequences = filter_out_sequences(partial_sequences)
print(partial_sequences)

        score                       pos_list
0  -22.812527      (, NIL, VBD, TO, VB, PPO)
1  -23.669951      (, NIL, VBN, TO, VB, PPO)
2  -20.606571       (, NN, VBD, TO, VB, PPO)
3  -22.967353       (, NN, VBN, TO, VB, PPO)
4  -22.682413    (, NN-TL, VBD, TO, VB, PPO)
5  -26.112298    (, NN-TL, VBN, TO, VB, PPO)
6  -19.002156       (, NP, VBD, TO, VB, PPO)
7  -24.197306       (, NP, VBN, TO, VB, PPO)
8  -10.379500     (, PPSS, VBD, TO, VB, PPO)
9  -18.494581     (, PPSS, VBN, TO, VB, PPO)
10 -22.119380  (, PPSS-HL, VBD, TO, VB, PPO)
11 -22.976804  (, PPSS-HL, VBN, TO, VB, PPO)
12 -19.634473  (, PPSS-NC, VBD, TO, VB, PPO)
13 -20.491897  (, PPSS-NC, VBN, TO, VB, PPO)
       score                       pos_list
0 -22.812527      (, NIL, VBD, TO, VB, PPO)
1 -20.606571       (, NN, VBD, TO, VB, PPO)
2 -22.682413    (, NN-TL, VBD, TO, VB, PPO)
3 -19.002156       (, NP, VBD, TO, VB, PPO)
4 -10.379500     (, PPSS, VBD, TO, VB, PPO)
5 -22.119380  (, PPSS-HL, VBD, TO, VB, PPO)
6 -19.634473  (, 

Once more.

In [28]:
# No need to compute a cross-product, since there is only one possible part of speech for a word out of range: ''
# Prepend a blank part-of-speech to all the part of speech sequences
partial_sequences['pos_list'] = partial_sequences.apply(lambda row: ('', *row['pos_list']), axis=1)

# Add contextual log probabilities to old
partial_sequences['score'] += partial_sequences['pos_list'].apply(compute_contextual_log_probability)

#print(partial_sequences)

# Filter out bad paths
partial_sequences = filter_out_sequences(partial_sequences)
print(partial_sequences)

     score                      pos_list
0 -10.3795  (, , PPSS, VBD, TO, VB, PPO)


And at long last, there we go. The optimal assignment is: I/PPSS tried/VBD to/TO please/VB you/PPO

I hope this in-depth answer pleases you.